## Getting Started with the Maxar Geospatial Platform Monitoring SDK

This sample workflow will guide a user through using the monitoring class of the Maxar Geospatial Platform SDK with real world examples. The process will go through:
* Validating a monitor
* Creating a monitor
* Listing monitors and monitor events
* Toggling a monitor between enable and disable

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the Monitoring portion of the Maxar Geospatial Platform SDK

In [ ]:
mgp_monitoring = interface.monitoring_service

#### Creating a new monitor

Before creating a monitor, some variables will be established to make the submission process easier. Note that monitors must be set to a time at least 15 minutes into the future.

In [ ]:
import datetime

now_start = datetime.datetime.utcnow() + datetime.timedelta(minutes=20)
now_time = now_start.isoformat()
end_start = datetime.datetime.utcnow() + datetime.timedelta(days=1)
end_time = end_start.isoformat()
now = now_time.replace(now_time[-7:], "+00:00")
end = end_time.replace(now_time[-7:], "+00:00")
print(now)
print(end)

The monitoring API utilizes GeoJSON formatts for an Area Of Interest (AOI).

In [ ]:
geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [-106.8, 35.1],
            [-106.4, 35.1],
            [-106.4, 35.4],
            [-106.8, 35.4],
            [-106.8, 35.1]
        ]
    ]
}

The match criteria will be set to find features taken by either WorldView 2 or WorldView 3 with a cloud cover less than 75%.

In [ ]:
criteria = {
    "platform": {
        "in": ["worldview-03", "worldview-02"]
    },
    "eo:cloud_cover": {
        "lt": 75
    },
    "aoi:coverage_sqkm": {
        "gte": 1
    }
}

When a monitor finds imagery over the desired AOI, a notification will be sent to the user. The below notification schema will be set to email when imagery matching the desired criteria is found.

In [ ]:
notifications = [
    {
        "type": "email",
        "target": "your.email@address.com"
    }
]

With the variables defined, the monitor request is almost ready to be sent. Imagery will be searched for in the discovery catalog. Firstly, the monitor request should be verified so that everything is constructed correctly with a verification call using the validate argument in the `new_monitor` function.

In [ ]:
monitor_validation = mgp_monitoring.new_monitor(source="discovery/catalog", start_datetime=now, end_datetime=end, 
                                                description="Your monitor description", aoi_geojson=geojson, 
                                                match_criteria=criteria, monitor_notifications=notifications, validate=True)
print(monitor_validation)

With the monitor passing the validation, the monitor request can now be submitted.

In [ ]:
monitor = mgp_monitoring.new_monitor(source="discovery/catalog", start_datetime=now, end_datetime=end, 
                                     description="Your monitor description", aoi_geojson=geojson, match_criteria=criteria, 
                                     monitor_notifications=notifications)
print(monitor)

#### Monitor details

With the monitor submission created, a user can now look at the details for the monitor by taking the monitor id and passing it into the `get_monitor` function.

In [ ]:
my_monitor = mgp_monitoring.get_monitor(monitor['data']['id'])
print(my_monitor)

After a monitor has been submitted, a user can look at the monitor events as they have taken place with the `get_monitor_events` function.

In [ ]:
monitor_events = mgp_monitoring.get_monitor_events(monitor['data']['id'])
print(monitor_events)

A user can also filter these events by using limit, filter, or sort as keyword arguments.

#### Toggle monitors

There may be times that a user wishes to disable an enabled monitor before the initial end time or enable a disabled monitor that was disabled prematurely. This can be done with the `toggle_monitor_status` function. *Note: Disabled monitors can only be enabled if the end datetime hasn't been reached yet. Enabled monitors can only be disabled if the start datetime has already happened*

In [ ]:
disable_monitor = mgp_monitoring.toggle_monitor_status(monitor_id=monitor['data']['id'], status="disable")
print(disable_monitor)

In [ ]:
enable_monitor = mgp_monitoring.toggle_monitor_status(monitor_id=monitor['data']['id'], status="enable")
print(enable_monitor)